In [41]:
#Load dependencies
import pandas as pd
import numpy as np
from statistics import mean
import statistics

In [2]:
#Load csv file
fantasy_df = pd.read_csv('Resources/purchase_data.csv')
fantasy_df.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


In [23]:
#Number of players
fantasy_df.count()
fantasy_df['SN'].unique()
num_of_players = len(fantasy_df['SN'].unique())
num_of_players

576

In [24]:
#number of rows in dataset
len(fantasy_df)

780

In [4]:
#Purchasing Analysis
num_of_items = len(fantasy_df['Item ID'].unique())
num_of_items

183

In [29]:
#Purchase Analysis 2
inventory_df = pd.DataFrame(fantasy_df[['Item ID', 'Price']].drop_duplicates(['Item ID']))
num_of_items = len(inventory_df)
num_of_items

183

In [30]:
prices = list(fantasy_df['Price'].unique())
avg_price = round(mean(prices),2)
avg_price

3.03

In [6]:
tot_num_of_purch = len(fantasy_df['Item ID'])
tot_num_of_purch

780

In [7]:
tot_rev = fantasy_df['Price'].sum()
tot_rev

2379.77

In [8]:
#Gender Demographics
#fantasy_df['Gender'].value_counts()
gender_demo = fantasy_df.drop_duplicates(['SN'])
gender_demo

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
773,773,Hala31,21,Male,19,"Pursuit, Cudgel of Necromancy",1.02
774,774,Jiskjask80,11,Male,101,Final Critic,4.19
775,775,Aethedru70,21,Female,60,Wolf,3.54
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46


In [9]:
gender_summary = pd.DataFrame(gender_demo['Gender'].value_counts())
gender_summary.reset_index()
gender_summary = gender_summary.rename(columns={'Gender': 'Counts'})
gender_summary

,Counts
Male,484
Female,81
Other / Non-Disclosed,11


In [10]:
gender_summary['Percentage(%)'] = round((gender_summary['Counts']/num_of_players)*100,2)
gender_summary

,Counts,Percentage(%)
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [13]:
purchase_analysis = fantasy_df.groupby('Gender').count()
df1 = purchase_analysis[['Item ID']]
df1 = df1.rename(columns={'Item ID': 'Purchase Count'})
df1

,Purchase Count
Gender,
Female,113
Male,652
Other / Non-Disclosed,15


In [35]:
fantasy_df.groupby('Gender', as_index=False).agg({'Item ID': 'count', 'Price': ['sum', lambda x: round(x.unique().mean(),2)]})

Gender Item ID    Price           
                           count      sum <lambda_0>
0                 Female     113   361.94       3.11
1                   Male     652  1967.64       3.04
2  Other / Non-Disclosed      15    50.19       3.31

In [55]:
fantasy_df.groupby('Gender').agg(
    Purchase_Count = ('Item ID', 'count'),
    Total_Revenue = ('Price', sum),
    Average_Price = ("Price", lambda x: sum(x))
)

KeyError: "[('Price', '<lambda>')] not in index"

In [69]:
fantasy_df.groupby('Gender').agg(
    Purchase_Count = pd.NamedAgg(column = "Item ID", aggfunc = 'count'),
    Average_Price = pd.NamedAgg(column = "Price", aggfunc = lambda x: round(x.unique().mean(),2)),
    #Total_Revenue = pd.NamedAgg(column = "Price", aggfunc = sum)
    
)

,Purchase_Count,Average_Price
Gender,,
Female,113,3.11
Male,652,3.04
Other / Non-Disclosed,15,3.31


In [63]:
male_df = fantasy_df[fantasy_df.Gender == 'Male']
male_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
...,...,...,...,...,...,...,...
774,774,Jiskjask80,11,Male,101,Final Critic,4.19
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46
778,778,Sisur91,7,Male,101,Final Critic,4.19


In [17]:
female_df = fantasy_df[fantasy_df.Gender== 'Female']
female_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
15,15,Lisassa64,21,Female,98,"Deadline, Voice Of Subtlety",2.89
18,18,Reunasu60,22,Female,82,Nirvana,4.90
38,38,Reulae52,10,Female,116,Renewed Skeletal Katana,4.18
41,41,Assosia88,20,Female,7,"Thorn, Satchel of Dark Souls",1.33
55,55,Phaelap26,25,Female,84,Arcane Gem,3.79
...,...,...,...,...,...,...,...
731,731,Eudanu84,22,Female,12,Dawne,1.02
740,740,Reunasu60,22,Female,127,"Heartseeker, Reaver of Souls",3.92
754,754,Pheosurllorin41,23,Female,79,"Alpha, Oath of Zeal",4.05
767,767,Ilmol66,8,Female,92,Final Critic,4.88


In [21]:
other_gender_df = fantasy_df[fantasy_df.Gender== 'Other / Non-Disclosed']
other_gender_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58
22,22,Siarithria38,38,Other / Non-Disclosed,24,Warped Fetish,3.81
82,82,Haerithp41,16,Other / Non-Disclosed,160,Azurewrath,4.40
111,111,Sundim98,21,Other / Non-Disclosed,41,Orbit,4.75
228,228,Jiskirran77,20,Other / Non-Disclosed,80,Dreamsong,3.39
237,237,Idairin51,20,Other / Non-Disclosed,16,Restored Bauble,3.55
242,242,Eodaisu60,16,Other / Non-Disclosed,39,"Betrayal, Whisper of Grieving Widows",3.94
291,291,Idairin51,20,Other / Non-Disclosed,35,Heartless Bone Dualblade,3.45
350,350,Rairith81,15,Other / Non-Disclosed,34,Retribution Axe,2.22
401,401,Lirtim36,15,Other / Non-Disclosed,46,Hopeless Ebon Dualblade,1.33
